# Demo: Teach an LLM a New Skill with SFT

Welcome! This notebook is a short demonstration to show you how to teach a Large Language Model (LLM) a new skill using Supervised Fine-Tuning (SFT). 

LLMs are great at many things, but they don't know everything. Sometimes, we need to teach them a specific, new task. In this demo, we'll teach a small LLM to add the suffish "-ish" to the ends of words.

This demo follows the exact same structure as the exercise you're about to do. Pay attention to the steps, as you'll be repeating them to teach the model how to spell.

## What you'll see in this demo

1.  **Setup**: Import libraries and configure the environment.
2.  **Load the model**: Use a small, instruction-tuned model as our starting point.
3.  **Create a dataset**: Generate a simple dataset of words and their -ish variants.
4.  **Evaluate the base model**: See how the model does *before* any training.
5.  **Configure LoRA and train**: Use Parameter-Efficient Fine-Tuning (PEFT) with LoRA to train our model efficiently.
6.  **Evaluate the fine-tuned model**: Test the model again to see its new skill in action.

## Setup

In [7]:
pip install trl

  Using cached trl-0.28.0-py3-none-any.whl.metadata (11 kB)
Using cached trl-0.28.0-py3-none-any.whl (540 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
# Setup necessary imports

import os
import torch
from datasets import Dataset

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer, SFTConfig

# Use GPU, MPS, or CPU, depending on what's available
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
torch.set_num_threads(max(1, os.cpu_count() // 2))
print(f"Using device: {device}")

Using device: cpu


## Step 1. Load the tokenizer and base model

We'll use `HuggingFaceTB/SmolLM2-135M-Instruct`, a small model with 135 million parameters. Its small size makes it perfect for a quick demonstration on a standard computer.

In [9]:
# Model ID from Hugging Face
model_id = "HuggingFaceTB/SmolLM2-135M-Instruct"

# Load the tokenizer, which prepares text for the model
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load the model itself
model = AutoModelForCausalLM.from_pretrained(model_id)

# Move the model to our selected device (GPU/CPU)
model = model.to(device)

print(f"Model loaded: {model_id}")

c:\Users\victor.francheto\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\victor.francheto\.cache\huggingface\hub\models--HuggingFaceTB--SmolLM2-135M-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading weights: 100%|██████████| 272/272 [00:00<00:00, 

Model loaded: HuggingFaceTB/SmolLM2-135M-Instruct


## Step 2. Create the dataset

Next, we'll create a small dataset of examples to teach the model our new task.

In [10]:
# A list of words for our demonstration

# fmt: off
DEMO_WORDS = [
    "idea", "glow", "rust", "maze", "echo", "wisp", "veto", "lush", "gaze", "knit", "fume", "plow",
    "void", "oath", "grim", "crisp", "lunar", "fable", "quest", "verge", "brawn", "elude", "aisle",
    "ember", "crave", "ivory", "mirth", "knack", "wryly", "onset", "mosaic", "velvet", "sphinx",
    "radius", "summit", "banner", "cipher", "glisten", "mantle", "scarab", "expose", "fathom",
    "tavern", "fusion", "relish", "lantern", "enchant", "torrent", "capture", "orchard", "eclipse",
    "frescos", "triumph", "absolve", "gossipy", "prelude", "whistle", "resolve", "zealous",
    "mirage", "aperture", "sapphire",
]
# fmt: on

print(f"Dataset will be created from {len(DEMO_WORDS)} words.")

Dataset will be created from 62 words.


In [11]:
# This function creates prompt/completion pairs for our dataset.
def generate_records():
    for word in DEMO_WORDS:
        # The prompt tells the model what to do.
        prompt = (
            f"Add -ish to the end of the word.\n"
            "hello -> hello-ish\n"
            "learn -> learn-ish\n"
            f"{word} -> "
        )
        # The completion is the correct answer.
        completion = f"{word}-ish"
        yield {"prompt": prompt, "completion": completion}


# Create a Hugging Face Dataset from our generator
ds = Dataset.from_generator(generate_records)

# Split the dataset: 80% for training, 20% for testing
ds = ds.train_test_split(test_size=0.2, seed=42)

# Let's look at the first training example
print("First training example:")
print(ds["train"][0])

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 62 examples [00:00, 14172.26 examples/s]

First training example:
{'prompt': 'Add -ish to the end of the word.\nhello -> hello-ish\nlearn -> learn-ish\nivory -> ', 'completion': 'ivory-ish'}


## Step 3. Evaluate the base model

Before we train, let's see if the model already knows how to do this.

In [12]:
# A helper function to test the model's translation ability
def check_translation(model, tokenizer, prompt: str, actual_translation: str):
    # Prepare the input for the model
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate a response from the model
    gen = model.generate(**inputs, max_new_tokens=15, use_cache=False)
    output = tokenizer.decode(gen[0], skip_special_tokens=True)

    # Extract just the translated part
    proposed_translation = output.split("->")[-1].strip().split("\n")[0].strip()

    # Check if the model's answer is correct
    is_correct = proposed_translation == actual_translation
    print(
        f"Proposed: {proposed_translation} | Actual: {actual_translation} "
        f"| Correct: {'✅' if is_correct else '❌'}"
    )
    return is_correct

In [13]:
print("--- Evaluating Base Model (Before Training) ---")
num_correct = 0
num_examples = len(ds["test"])

for example in ds["test"]:
    prompt = example["prompt"]
    completion = example["completion"]
    if check_translation(model, tokenizer, prompt, completion):
        num_correct += 1

print(f"\nResult: {num_correct}/{num_examples} correct.")

--- Evaluating Base Model (Before Training) ---
Proposed: wryly-ish | Actual: wryly-ish | Correct: ✅
Proposed: 1-ish | Actual: glisten-ish | Correct: ❌
Proposed: quest-ish | Actual: quest-ish | Correct: ✅
Proposed: ire-ish | Actual: crave-ish | Correct: ❌
Proposed: ils-ish | Actual: lush-ish | Correct: ❌
Proposed: файлей | Actual: fable-ish | Correct: ❌
Proposed: knack-ish | Actual: knack-ish | Correct: ✅
Proposed: iumph-ish | Actual: triumph-ish | Correct: ❌
Proposed: sapphire-ish | Actual: sapphire-ish | Correct: ✅
Proposed: expose-ish | Actual: expose-ish | Correct: ✅
Proposed: ils-es | Actual: frescos-ish | Correct: ❌
Proposed: wisp-ish | Actual: wisp-ish | Correct: ✅
Proposed: mi-rage | Actual: mirage-ish | Correct: ❌

Result: 6/13 correct.


The base model does OK, but it can do better.

## Step 4. Configure LoRA and train the model

We'll use Low-Rank Adaptation (LoRA) to make training fast and memory-efficient. LoRA adds a small number of new, trainable parameters to the model, freezing the original ones. This means we only have to update a tiny fraction of the model's weights.

In [14]:
# LoRA configuration
lora_config = LoraConfig(
    r=64,  # Rank of the update matrices. Lower is fewer parameters.
    lora_alpha=16,  # LoRA scaling factor. Generally set to 16.
    lora_dropout=0.05,  # Dropout for LoRA layers
    bias="none",
    task_type="CAUSAL_LM",
)

# Wrap the base model with LoRA layers
model = get_peft_model(model, lora_config)

# Print the percentage of trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(
    f"Trainable params: {trainable_params:,} / {total_params:,} ({100 * trainable_params / total_params:.2f}%)"
)

Trainable params: 3,686,400 / 138,201,408 (2.67%)


Notice that we're only training about 2.67% of the total parameters! Now, we set the training arguments.

In [16]:
# Training arguments for the SFTTrainer
training_args = SFTConfig(
    output_dir="data/model_demo",  # Directory to save artifacts
    per_device_train_batch_size=8,  # Small batch size for demo
    gradient_accumulation_steps=2,  # Two forward and backward passes per update step
    num_train_epochs=20,  # Number of times to go through the data
    learning_rate=2e-4,  # Controls how much the model weights are updated
    logging_steps=50,  # Log training progress every 10 steps
    save_strategy="no",  # Don't save model checkpoints
    report_to=[],  # Disable reporting to services like Weights & Biases
    fp16=False,  # Use full precision (fp32) for wider compatibility
)

ValueError: Your setup doesn't support bf16/gpu. You need to assign use_cpu if you want to train the model on CPU

In [ ]:
# Create the SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    args=training_args,
)

# Start the training process!
print("--- Starting Training ---")
trainer.train()
print("--- Training Complete ---")

--- Starting Training ---


  0%|          | 0/60 [00:00<?, ?it/s]

/Users/briancruz/udacity/udacity genai nd c1 project refresh 2025/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 0.3888, 'grad_norm': 0.05742060765624046, 'learning_rate': 3.3333333333333335e-05, 'num_tokens': 21162.0, 'mean_token_accuracy': 0.9018999320268631, 'epoch': 14.29}
{'train_runtime': 14.9863, 'train_samples_per_second': 65.393, 'train_steps_per_second': 4.004, 'train_loss': 0.32870734483003616, 'num_tokens': 25352.0, 'mean_token_accuracy': 1.0, 'epoch': 17.14}
--- Training Complete ---


## Step 5. Evaluate the fine-tuned model

Training is done! Now for the moment of truth. Let's see if our model learned the task.

In [ ]:
print("--- Evaluating Fine-Tuned Model (After Training) ---")
num_correct = 0
num_examples = len(ds["test"])

for example in ds["test"]:
    prompt = example["prompt"]
    completion = example["completion"]
    if check_translation(model, tokenizer, prompt, completion):
        num_correct += 1

print(f"\nResult: {num_correct}/{num_examples} correct.")

--- Evaluating Fine-Tuned Model (After Training) ---
Proposed: wryly-ish | Actual: wryly-ish | Correct: ✅
Proposed: 1-ish | Actual: glisten-ish | Correct: ❌
Proposed: quest-ish | Actual: quest-ish | Correct: ✅
Proposed: crave-ish | Actual: crave-ish | Correct: ✅
Proposed: lus-ish | Actual: lush-ish | Correct: ❌
Proposed: fable-ish | Actual: fable-ish | Correct: ✅
Proposed: knack-ish | Actual: knack-ish | Correct: ✅
Proposed: t-m-i-p-h-e-l-o | Actual: triumph-ish | Correct: ❌
Proposed: sapphire-ish | Actual: sapphire-ish | Correct: ✅
Proposed: expose-ish | Actual: expose-ish | Correct: ✅
Proposed: Frescos-ish | Actual: frescos-ish | Correct: ❌
Proposed: wisp-ish | Actual: wisp-ish | Correct: ✅
Proposed: mirage-ish | Actual: mirage-ish | Correct: ✅

Result: 9/13 correct.


## Conclusion 🎉

Success! After a very short training run on a tiny dataset, the model improved on the task. It went from 7/13 to 9/13, which is modest, but shows fine-tuning with parameter-efficient fine-tuning works.

<br /><br /><br /><br /><br /><br /><br /><br />